Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

Set Device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameters

In [3]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

Load Data

In [4]:
train_dataset = datasets.MNIST(root='./datatset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,)
test_dataset = datasets.MNIST(root='./datatset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True,)

Create Fully Connected Network

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(in_features=hidden_size*sequence_length, out_features=num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        #Forward Propagation
        out, _ = self.rnn(x,h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)

        return out

Initialize the Network

In [6]:
model = RNN(input_size=input_size,num_classes=num_classes, hidden_size=hidden_size, num_layers=num_layers).to(device)

Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Train the Network

In [8]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for batch_idx ,(data, target) in enumerate(train_loader):
        #Get data to cuda
        data = data.to(device=device).squeeze(1)
        target = target.to(device=device)
        
        #Get to correct shape
        #data = data.reshape(data.shape[0], -1)

        #forward
        scores = model(data)
        loss = criterion(scores, target)

        #Backward
        optimizer.zero_grad()
        loss.backward()

        #Gradient Descent or Adam Step
        optimizer.step()
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, batch_idx+1, total_step, loss.item()))

Epoch [1/2], Step [1/938], Loss: 2.3064
Epoch [1/2], Step [2/938], Loss: 2.3869
Epoch [1/2], Step [3/938], Loss: 2.2599
Epoch [1/2], Step [4/938], Loss: 2.1011
Epoch [1/2], Step [5/938], Loss: 1.9615
Epoch [1/2], Step [6/938], Loss: 1.6223
Epoch [1/2], Step [7/938], Loss: 1.5092
Epoch [1/2], Step [8/938], Loss: 1.3050
Epoch [1/2], Step [9/938], Loss: 1.2690
Epoch [1/2], Step [10/938], Loss: 1.0497
Epoch [1/2], Step [11/938], Loss: 0.7948
Epoch [1/2], Step [12/938], Loss: 0.8340
Epoch [1/2], Step [13/938], Loss: 1.0050
Epoch [1/2], Step [14/938], Loss: 1.0238
Epoch [1/2], Step [15/938], Loss: 0.6996
Epoch [1/2], Step [16/938], Loss: 0.6797
Epoch [1/2], Step [17/938], Loss: 0.6685
Epoch [1/2], Step [18/938], Loss: 0.5645
Epoch [1/2], Step [19/938], Loss: 0.6442
Epoch [1/2], Step [20/938], Loss: 0.8347
Epoch [1/2], Step [21/938], Loss: 0.7379
Epoch [1/2], Step [22/938], Loss: 0.7113
Epoch [1/2], Step [23/938], Loss: 0.8042
Epoch [1/2], Step [24/938], Loss: 0.3100
Epoch [1/2], Step [25/938

Check Accuracy

In [9]:
def evaluate(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples=0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _,pred = scores.max(1)
            num_correct += (pred == y).sum()
            num_samples += pred.size(0)

        print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
    model.train()

In [10]:
evaluate(train_loader, model)
evaluate(test_loader, model)

Checking accuracy on training data
Got 58089/60000 with accuracy 96.81
Checking accuracy on test data
Got 9650/10000 with accuracy 96.50
